#### Importing the libaries 

In [1]:
import pandas as pd
import numpy as np

In [2]:
loan_df = pd.read_csv("data/loan.csv",low_memory = False)

#### The loan dataset contains 39717 rows and 111 columns, where the loan status is the dependent variable and the rest are independent variables. we need to build a ML classfication model. Before building the model we need to preprocess the data for better results.

In [3]:
loan_df.shape

(39717, 111)

In [4]:
loan_df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)

In [5]:
loan_df['loan_status'].value_counts()    # dependent variable   # target variable

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

#### there are 3 classes in the target variable

#### missing value handling

In [6]:
loan_df.isna().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
                              ...  
tax_liens                        39
tot_hi_cred_lim               39717
total_bal_ex_mort             39717
total_bc_limit                39717
total_il_high_credit_limit    39717
Length: 111, dtype: int64

In [7]:
loan_df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)

#### identifying the columns where the missing values are more than 15 percent

In [8]:
miss_columns_per = (loan_df.isna().sum() / loan_df.shape[0] *100) > 15   ### 10 percent

In [9]:
missing_columns = loan_df.columns[miss_columns_per]

#### dropping the particular columns where the missing values are more than15 percent

In [10]:
loan_df2 = loan_df.drop(missing_columns,axis=1)

In [11]:
loan_df2.isna().sum() / loan_df.shape[0] * 100

id                            0.000000
member_id                     0.000000
loan_amnt                     0.000000
funded_amnt                   0.000000
funded_amnt_inv               0.000000
term                          0.000000
int_rate                      0.000000
installment                   0.000000
grade                         0.000000
sub_grade                     0.000000
emp_title                     6.191303
emp_length                    2.706650
home_ownership                0.000000
annual_inc                    0.000000
verification_status           0.000000
issue_d                       0.000000
loan_status                   0.000000
pymnt_plan                    0.000000
url                           0.000000
purpose                       0.000000
title                         0.027696
zip_code                      0.000000
addr_state                    0.000000
dti                           0.000000
delinq_2yrs                   0.000000
earliest_cr_line         

#### checking with emp_title

In [12]:
loan_df2['emp_title'].nunique()

28820

#### there are 28820 categories in that. which is not going to be helpful in ML model. hence drop it

In [13]:
loan_df3 = loan_df2.drop("emp_title",axis=1)

#### rest of the null values i want to drop them

In [14]:
loan_df4 = loan_df3.dropna()

#### em_length column

In [15]:
loan_df3['emp_length'].value_counts()

10+ years    8879
< 1 year     4583
2 years      4388
3 years      4095
4 years      3436
5 years      3282
1 year       3240
6 years      2229
7 years      1773
8 years      1479
9 years      1258
Name: emp_length, dtype: int64

In [16]:
loan_df3.shape

(39717, 52)

#### id and member_id columns are just identification numbers. so not going to be helpful in ML model

In [17]:
loan_df5 = loan_df4.drop(['id','member_id'],axis=1)

#### lets handle the term column. it contains text values, so need to convert into numerical, lets apply some regex to extract the numerical values

In [18]:
import re

In [19]:
loan_df5['term'] = loan_df5['term'].apply(lambda x: re.findall("\d+",str(x))[0])

In [20]:
# loan_df5.dtypes

#### lets convert the object data type into numerical

In [21]:
loan_df5['term'] = loan_df5['term'].astype(int)

#### lets deal with the int_rate column, it is also given the object data type. lets remove the percentage value

In [22]:
loan_df5['int_rate'] = loan_df5['int_rate'].str.replace("%","").astype(float)

#### lets deal with grade column

In [23]:
loan_df5['grade'].unique()      # i want to do the encoding here

array(['B', 'C', 'A', 'E', 'F', 'D', 'G'], dtype=object)

#### we can do the label encoding or one hot encoding for this

#### lets deal with emo_length column. extract the numeric value using regex

In [24]:
loan_df5['emp_length'] = loan_df5['emp_length'].apply(lambda x: re.findall("\d+",str(x))[0]).astype(int)

#### lets deal with the home_ownership column

In [25]:
loan_df5['home_ownership'].unique()    # one hot encoding

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER'], dtype=object)

#### it has 4 categories so we can do the one hot encoding for that

#### lets check with verification status. it has 3 categories so u can do one hot encoding

In [26]:
loan_df5['verification_status'].unique()     # one hot encoding

array(['Verified', 'Source Verified', 'Not Verified'], dtype=object)

#### lets dela with pymnt_plan column.

In [27]:
loan_df5['pymnt_plan'].unique()

array(['n'], dtype=object)

#### it has only one single variable, so drop it

In [28]:
loan_df6 = loan_df5.drop("pymnt_plan",axis=1)

#### url column is also not giving any info for the model building , so drop it

In [29]:
loan_df7 = loan_df6.drop("url",axis=1)

#### lets deal with zip_code.

In [30]:
loan_df7['zip_code'].nunique()

808

#### it has 808 categories. you cand drop it or handle ot properly

In [31]:
loan_df8 = loan_df7.drop('zip_code',axis=1)

#### lets deal with addr_state column

In [32]:
loan_df8['addr_state'].unique()      # one hot encoding

array(['AZ', 'GA', 'IL', 'CA', 'OR', 'NC', 'TX', 'VA', 'MO', 'CT', 'UT',
       'FL', 'PA', 'MN', 'NY', 'NJ', 'KY', 'OH', 'SC', 'RI', 'LA', 'MA',
       'WA', 'WI', 'AL', 'CO', 'KS', 'NV', 'AK', 'MD', 'WV', 'VT', 'MI',
       'DC', 'SD', 'NH', 'AR', 'NM', 'MT', 'HI', 'WY', 'OK', 'DE', 'MS',
       'TN', 'IA', 'NE', 'ID'], dtype=object)

#### we can go for one hot encoding

#### lets see the issue_d column

In [33]:
loan_df8['issue_d']

0        Dec-11
1        Dec-11
2        Dec-11
3        Dec-11
4        Dec-11
          ...  
39475    Dec-07
39551    Dec-07
39562    Nov-07
39573    Nov-07
39623    Oct-07
Name: issue_d, Length: 37823, dtype: object

#### it has date column so extract the month and year column and drop the issue_d

In [34]:
from datetime import datetime

In [35]:
loan_df8['issue_d'] = loan_df8['issue_d'].apply(lambda x: datetime.strptime(x,"%b-%y"))

In [36]:
#### feature engineering

In [37]:
loan_df8['month'] = loan_df8['issue_d'].apply(lambda x:x.month)

In [38]:
loan_df8['year'] = loan_df8['issue_d'].apply(lambda x:x.year)

In [39]:
loan_df9 = loan_df8.drop(["issue_d","application_type"],axis=1)

#### lets deal with the revol_util column. remove the percentage symbol and convert the data type to numeric

In [40]:
loan_df9['revol_util'] = loan_df9['revol_util'].str.replace("%","").astype(float)

#### lets deal with initial_list_status

In [41]:
loan_df9['initial_list_status'].nunique()

1

#### it has only one category so drop it

In [42]:
loan_df10  = loan_df9.drop("initial_list_status",axis=1)

#### the last_pymnt_d and the last_credit_pull_d are the date columns. so extract the month and year and drop the original columns

In [43]:
loan_df8['last_pymnt_d'] = loan_df8['last_pymnt_d'].apply(lambda x: datetime.strptime(x,"%b-%y"))
loan_df8['last_credit_pull_d'] = loan_df8['last_credit_pull_d'].apply(lambda x: datetime.strptime(x,"%b-%y"))

In [44]:
loan_df8['lp_month'] = loan_df8['last_pymnt_d'].apply(lambda x:x.month)
loan_df8['lp_year'] = loan_df8['last_pymnt_d'].apply(lambda x:x.year)

In [45]:
loan_df8['lc_month'] = loan_df8['last_credit_pull_d'].apply(lambda x:x.month)
loan_df8['lc_year'] = loan_df8['last_credit_pull_d'].apply(lambda x:x.year)

In [46]:
loan_df11 = loan_df10.drop(['last_pymnt_d','last_credit_pull_d'],axis=1)

#### dropping the earliest_cr_line

In [47]:
loan_df12 = loan_df11.drop('earliest_cr_line',axis=1)

#### dropping tne title column

In [48]:
loan_df13 = loan_df12.drop('title',axis=1)

#### store the independent variables into x and dependent variable to y

In [49]:
x = loan_df13.drop("loan_status",axis=1)
y = loan_df13['loan_status']

#### do the one hot encoding for the rest of the object columns

In [50]:
x = pd.get_dummies(x)

#### devide the data into train. test

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=43)

In [53]:
num_cols = x_train.columns[:35]

In [54]:
x_train.columns[:35]

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'emp_length', 'annual_inc', 'dti', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens', 'month', 'year'],
      dtype='object')

### standard scaling

In [55]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

x_train.loc[:,num_cols] = sc.fit_transform(x_train[num_cols])
x_test.loc[:,num_cols] = sc.fit_transform(x_test[num_cols])